In [9]:
from google.cloud import bigquery
import pandas as pd
import numpy as np
#import pandas_gbp
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime
from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)
import import_ipynb
import covid_bigquery as cb
import plotly.express as px

import plotly.tools as tls
from plotly.offline import iplot
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
init_notebook_mode(connected=True)

def fetch_exchange_rates():
    # Fetch exchange rate
    df_exchange_rates = pd.read_excel('exchangerates.xlsx')
    #df_exchange_rates['Date'] = df_exchange_rates['Date'].map(lambda x: datetime.strptime(x, "%d/%m/%Y"))
    df_exchange_rates['Rate'] = df_exchange_rates['Rate'].map(lambda x: float(x))
    return df_exchange_rates
df_exchange_rates = fetch_exchange_rates()

covid19 = cb.Covid19()

In [ ]:
# Plots ratios calculated above, need to change this to pass column names
def covid_plot_ratio(df_covid, df_exchange_rates, country1, country2, type, exch_ratelabel):
    fig, ax = plt.subplots(figsize=(16,4))
    ax.plot(df_covid['date'], df_covid['ratio'], label="{0}/{1} {2}".format(country1, country2, type))
    df_tmp = df_exchange_rates[df_exchange_rates['code'] == exch_ratelabel]
    ax.plot(df_tmp['date'], df_tmp['rate'], label=exch_ratelabel)
    #ax.set_xlim(0, 200) - doesnt work with dates
    #ax.set_ylim(0, 10)
    ax.xaxis.set_major_locator(MultipleLocator(30))
    ax.xaxis.set_minor_locator(AutoMinorLocator(7))
    ax.grid(which='major', color='#CCCCCC', linestyle='--')
    ax.grid(which='minor', color='#CCCCCC', linestyle=':')
    ax.legend()
    #plt.show()
    iplot(tls.mpl_to_plotly(fig))

def plot_covid_forex(covid19, df_exchange_rates, exchrate_code, country1, country2):
    df_ratio = covid19.covid_ratio_by_country('confirmed_per100k',   country1, country2)
    covid_plot_ratio(df_ratio, df_exchange_rates, country1, country2, 'confirmed', exchrate_code)

    df_ratio = covid19.covid_ratio_by_country('deaths_per100k',  country1, country2)
    covid_plot_ratio(df_ratio, df_exchange_rates, country1, country2, 'deaths', exchrate_code)

    df_ratio = covid19.covid_ratio_by_country('active_per100k',  country1, country2)
    covid_plot_ratio(df_ratio, df_exchange_rates, country1, country2, 'active', exchrate_code)

#df_exchange_rates['rate'][df_exchange_rates['code'] == exchrate_label] = df_exchange_rates['rate'][df_exchange_rates['code'] == exchrate_label]/35
plot_covid_forex(covid19, df_exchange_rates, 'USDINR', 'US', 'India')
plot_covid_forex(covid19, df_exchange_rates, 'GBPUSD', 'US', 'United Kingdom')


In [ ]:
plot_covid_forex(covid19, df_exchange_rates, 'EURUSD', 'Germany', 'US')
plot_covid_forex(covid19, df_exchange_rates, 'EURUSD', 'France', '')

In [10]:
def lineplot_by_country(
        dataframe, 
        y_axis,
        x_axis = "date",
        query = "country in ('US', 'United Kingdom', 'France', 'Italy', 'Egypt', 'India', 'China', 'Russia', 'South Africa') ", 
        color='country', 
        width=1200,
        log_y = False):
    fig = px.line(
            dataframe.query(query),
            x=x_axis, y=y_axis, color=color, width=width, log_y=log_y)
    fig.show()

lineplot_by_country(covid19.df_cumulative_by_country, covid19.COL_CONFIRMED, log_y = False)
lineplot_by_country(covid19.df_cumulative_by_country, covid19.COL_DEATHS, log_y = True)
lineplot_by_country(covid19.df_cumulative_by_country, covid19.COL_RECOVERED, log_y = True)
lineplot_by_country(covid19.df_cumulative_by_country, covid19.COL_CONFIRMED + "_per1m")
lineplot_by_country(covid19.df_cumulative_by_country, covid19.COL_DEATHS + "_per1m")
lineplot_by_country(covid19.df_cumulative_by_country, covid19.COL_RECOVERED + "_per1m")

In [12]:
def lineplot_by_region(
        dataframe, 
        y_axis,
        x_axis = "date",
        query = "country in ('China') ", 
        color = 'province_state', 
        width = 1200):
    fig = px.line(
            dataframe.query(query),
            x=x_axis, y=y_axis, color=color, width=width)
    fig.show()
dftemp = covid19.df_cumulative_raw[covid19.df_cumulative_raw['country']=='China']
lineplot_by_region(dftemp, covid19.COL_CONFIRMED)
lineplot_by_region(dftemp, covid19.COL_DEATHS)
lineplot_by_region(dftemp, covid19.COL_CONFIRMED + covid19.SUFFIX_100K)
lineplot_by_region(dftemp, covid19.COL_DEATHS + covid19.SUFFIX_100K)

In [42]:
import plotly.graph_objects as go
def areaplot_by_country(
        dataframe, 
        y_axis,
        x_axis = "date",
        y_max = .25,
        query = "country in ('US', 'United Kingdom', 'Italy') ", 
        color='country', 
        width=1200,
        log_y = False):
    fig = px.bar(
            dataframe.query(query),
            x=x_axis, y=y_axis, color=color, width=width, log_y=log_y)
    #fig.add_trace(x=[0], y=[datetime(2020, 3, 23)], text=["UK Lockdown"])
    fig.add_trace(go.Scatter(
        y=[0, y_max, 0],
        x=[datetime(2020, 3, 9), datetime(2020, 3, 9)],
        mode="lines+markers+text",
        hovertext=["Italy Lockdown", "Italy Lockdown"]
    ))
    fig.add_trace(go.Scatter(
        y=[0, y_max],
        x=[datetime(2020, 3, 23), datetime(2020, 3, 23)],
        mode="lines+markers+text",
        hovertext=["UK Lockdown", "UK Lockdown"]
    ))
    fig.add_trace(go.Scatter(
        y=[0, y_max],
        x=[datetime(2020, 3, 24), datetime(2020, 3, 24)],
        mode="lines+markers+text",
        hovertext=["US Lockdown", "US Lockdown"]
    ))
    fig.show()

areaplot_by_country(covid19.df_daily, covid19.COL_DAILY_CASES + '_per1k')
areaplot_by_country(covid19.df_daily, covid19.COL_DAILY_DEATHS+ '_per1k', y_max=.03)
areaplot_by_country(covid19.df_daily, covid19.COL_DAILY_CASES)
areaplot_by_country(covid19.df_daily, covid19.COL_DAILY_DEATHS)

In [ ]:
dftemp = covid19.df_cumulative_raw[covid19.df_cumulative_raw['country']=='United Kingdom']
lineplot_by_region(dftemp, covid19.COL_CONFIRMED)
lineplot_by_region(dftemp, covid19.COL_DEATHS)
lineplot_by_region(dftemp, covid19.COL_CONFIRMED + covid19.SUFFIX_100K)
lineplot_by_region(dftemp, covid19.COL_DEATHS + covid19.SUFFIX_100K)

In [ ]:
covid19.df_daily
lineplot_by_region(dftemp, covid19.COL_CONFIRMED)


In [ ]:
china = covid19.df_cumulative_raw[covid19.df_cumulative_raw[covid19.COL_COUNTRY] == 'China'].groupby([covid19.COL_COUNTRY, covid19.COL_DATE])

In [ ]:
china = china[covid19.COL_CONFIRMED, covid19.COL_DEATHS, covid19.COL_RECOVERED, covid19.COL_ACTIVE].sum().join(china[covid19.COL_POPULATION, 'country_code'].first())


In [ ]:
china.reset_index(level=0, inplace=True)
china[covid19.COL_PROVINCE] = 'ALL'


In [36]:
covid19.df_cumulative_raw[covid19.df_cumulative_raw['country']=='Italy']

,country,province_state,date,confirmed,deaths,recovered,active,country_code,population,confirmed_per1m,confirmed_per100k,deaths_per1m,deaths_per100k,recovered_per1m,recovered_per100k,active_per1m,active_per100k
260.0,Italy,ALL,2020-01-22,0.0,0.0,0.0,0.0,IT,62402659.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
526.0,Italy,ALL,2020-01-23,0.0,0.0,0.0,0.0,IT,62402659.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
792.0,Italy,ALL,2020-01-24,0.0,0.0,0.0,0.0,IT,62402659.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1058.0,Italy,ALL,2020-01-25,0.0,0.0,0.0,0.0,IT,62402659.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1324.0,Italy,ALL,2020-01-26,0.0,0.0,0.0,0.0,IT,62402659.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35904.0,Italy,ALL,2020-06-04,234013.0,33689.0,161895.0,38429.0,IT,62402659.0,3750.048536,375.004854,539.864816,53.986482,2594.360602,259.436060,615.823117,61.582312
36170.0,Italy,ALL,2020-06-05,234531.0,33774.0,163781.0,36976.0,IT,62402659.0,3758.349464,375.834946,541.226937,54.122694,2624.583674,262.458367,592.538853,59.253885
36436.0,Italy,ALL,2020-06-06,234801.0,33846.0,165078.0,35877.0,IT,62402659.0,3762.676203,376.267620,542.380734,54.238073,2645.368044,264.536804,574.927424,57.492742
36702.0,Italy,ALL,2020-06-07,234998.0,33899.0,165837.0,35262.0,IT,62402659.0,3765.833119,376.583312,543.230057,54.323006,2657.530988,265.753099,565.072075,56.507207


In [94]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
def barplot_by_country(
        dataframe, 
        y_axis,
        lockdown_start: datetime,
        x_axis = "date",
        y_max = .015,
        country = "Italy", 
        color='country', 
        width=1200,
        log_y = False):
    fig = px.bar(
            dataframe.query("country in ('{0}')".format(country)),
            x=x_axis, y=y_axis, color=color, width=width, log_y=log_y)
    fig.add_trace(go.Scatter(
        y=[0, y_max, 0],
        x=[lockdown_start, lockdown_start],
        mode="lines+markers+text",
        hovertext=[ "{0} Lockdown".format(country), "{0} Lockdown".format(country)]
    ), 1,1)
    fig.show()
barplot_by_country(covid19.df_daily, covid19.COL_DAILY_DEATHS + '_per1k', country='Italy', lockdown_start = datetime(2020, 3, 9))
barplot_by_country(covid19.df_daily, covid19.COL_DAILY_DEATHS + '_per1k', country='United Kingdom', lockdown_start = datetime(2020, 3, 23))
barplot_by_country(covid19.df_daily, covid19.COL_DAILY_DEATHS + '_per1k', country='New Zealand', lockdown_start = datetime(2020, 3, 26), y_max=0.002)
barplot_by_country(covid19.df_daily, covid19.COL_DAILY_CASES + '_per1k', country='Italy', lockdown_start = datetime(2020, 3, 9), y_max=0.12)
barplot_by_country(covid19.df_daily, covid19.COL_DAILY_CASES + '_per1k', country='United Kingdom', lockdown_start = datetime(2020, 3, 23), y_max=0.12)
barplot_by_country(covid19.df_daily, covid19.COL_DAILY_CASES + '_per1k', country='New Zealand', lockdown_start = datetime(2020, 3, 26), y_max=0.12)
#areaplot_by_country(covid19.df_daily, covid19.COL_DAILY_DEATHS+ '_per1k', y_max=.03)
#areaplot_by_country(covid19.df_daily, covid19.COL_DAILY_CASES)
#areaplot_by_country(covid19.df_daily, covid19.COL_DAILY_DEATHS)